CS5542 Big Data Apps and Analytics ICP -1

Spark Programming:  

Write a spark program to group the words in a given text file based on their starting letters.




Spark Integration with Google Research Colab


Spark is written in the Scala programming language and requires the Java Virtual Machine (JVM) to run. Therefore, our first task is to download Java.

Then Spark will be installed.

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.0.3/spark-3.0.3-bin-hadoop3.2.tgz
!tar xf spark-3.0.3-bin-hadoop3.2.tgz
!pip install -q findspark

* JVM Installed status and default java setting will be applied to run the PySpark program

In [ ]:
!ls /usr/lib/jvm/

default-java		   java-11-openjdk-amd64     java-8-openjdk-amd64
java-1.11.0-openjdk-amd64  java-1.8.0-openjdk-amd64


import os 
* it helps to create or identifying the directories in local setup

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.3-bin-hadoop3.2"

PySpark Installation check after the JVM and Spark installation.

In [ ]:
!pip install pyspark==3.0.2

     |████████████████████████████████| 204.8 MB 62 kB/s 
     |████████████████████████████████| 198 kB 39.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.2-py2.py3-none-any.whl size=205186690 sha256=1a890bd7a5ce0a116989e15852a36d7accf0e2f3352007627cb745e2c17109b2
  Stored in directory: /root/.cache/pip/wheels/9a/39/f6/970565f38054a830e9a8593f388b36e14d75dba6c6fdafc1ec
Successfully built pyspark


findspark will help to find the installed python version and package detailed about PySpark.

In [ ]:
import findspark
findspark.init()

In [ ]:
findspark.find()

'/usr/local/lib/python3.7/dist-packages/pyspark'

A SparkContext represents the connection to a Spark cluster, and can be used to create RDDs, accumulators and broadcast variables on that cluster. 

It can be only one SparkContext should be active per JVM. 

Existingactive SparkContext should be stopped by using stop() before creating a new one.

SparkConf allows configure  properties like master URL and application name)


In [ ]:
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName("Big_data_ICP_1").setMaster("local[*]")
sc = SparkContext.getOrCreate(conf = conf)

files.upload 
--> returns a dictionary of the files which were uploaded. 

The dictionary is keyed by the file name and values are the data which were uploaded.

In [ ]:
from google.colab import files
files.upload()

Saving icp.txt to icp.txt


{'icp.txt': b'As the Labor Day holiday nears, many people are planning travel and get-togethers to see family and friends.Unfortunately, this is occurring at the same time Covid-19 rates are climbing. The rates of new coronavirus infections are higher than they have been since January. Hospitalizations are also at their highest levels since January. In many parts of the United States, both infections and hospitalizations are higher than they were during Labor Day weekend in 2020.How should people think about Covid-19 safety now, compared to last year? Is it safe to see family and friends? What if extended family members want to stay in a house together -- what are some steps they should take to reduce risk? And how does the start of school affect our risk?To help navigate these questions, we spoke with CNN Medical Analyst Dr.Leana Wen. Wen is an emergency physician and visiting professor of health policy and management at the George Washington University Milken Institute School of Publ

Text File Data stored in input_text_file

In [ ]:
input_text_file = sc.textFile("icp.txt")

Count returns the number of elements in this RDD.

In [ ]:
input_text_file.count()

1

Takes the first num elements of the RDD stored during the session.

In [ ]:
input_text_file.take(1)

['As the Labor Day holiday nears, many people are planning travel and get-togethers to see family and friends.Unfortunately, this is occurring at the same time Covid-19 rates are climbing. The rates of new coronavirus infections are higher than they have been since January. Hospitalizations are also at their highest levels since January. In many parts of the United States, both infections and hospitalizations are higher than they were during Labor Day weekend in 2020.How should people think about Covid-19 safety now, compared to last year? Is it safe to see family and friends? What if extended family members want to stay in a house together -- what are some steps they should take to reduce risk? And how does the start of school affect our risk?To help navigate these questions, we spoke with CNN Medical Analyst Dr.Leana Wen. Wen is an emergency physician and visiting professor of health policy and management at the George Washington University Milken Institute School of Public Health. S

The below fucntion takes the text line as input and translates sentences into words by removing the punctuations '!"#$%&\'()*+,-./:;<=>...' 

strip will trim the spaces 

and function returns the final line when ever called from the execution

In [ ]:
import string
def StringFuncTranslate(textLine):
  line = textLine.translate(textLine.maketrans('', '',string.punctuation))
  line = line.strip()
  return line

PySpark flatMap() is a transformation operation that flattens the RDD after applying the function on every element and returns a new PySpark RDD.

Here the operation to read the entire line and pass to the above defined function to remove the punctuations and trim the spaces.

next the lambda is transformation function which will store the each word from the RDD and store it in new RDD. It is on the intial RDD.

In [ ]:
sp_rdd = input_text_file.flatMap(lambda line: StringFuncTranslate(line).split())

In [ ]:
sp_rdd.count()

245

In [ ]:
sp_rdd.take(10)

['As',
 'the',
 'Labor',
 'Day',
 'holiday',
 'nears',
 'many',
 'people',
 'are',
 'planning']

* Numeric data is removed the exisiting RDD so that it won't have any numeric values when grouping the words as RDD

In [ ]:
sp_rdd_alpha = sp_rdd.filter(lambda x: not x.isnumeric())

In [ ]:
sp_rdd_alpha.count()

244

capitalize --> Will help in changing the words to upper case (the starting alphabet of the word only)

In [ ]:
sp_rdd_cap = sp_rdd_alpha.map(lambda x: x.capitalize())

In [ ]:
sp_rdd_cap.take(10)

['As',
 'The',
 'Labor',
 'Day',
 'Holiday',
 'Nears',
 'Many',
 'People',
 'Are',
 'Planning']

Using map on the RDD to attach the key value pairs of words with starting letter and word as value.

In [ ]:
sp_rdd_word = sp_rdd_cap.map(lambda x: (x[0],x))

In [ ]:
sp_rdd_word.count()

244

In [ ]:
sp_rdd_word.take(10)

[('A', 'As'),
 ('T', 'The'),
 ('L', 'Labor'),
 ('D', 'Day'),
 ('H', 'Holiday'),
 ('N', 'Nears'),
 ('M', 'Many'),
 ('P', 'People'),
 ('A', 'Are'),
 ('P', 'Planning')]

* The map on the RDD may have duplicates, so distinct method is used to remove the duplicates and stored in a new rdd.

In [ ]:
sp_rdd_dist = sp_rdd_word.distinct()

In [ ]:
sp_rdd_dist.count()

150

In [ ]:
sp_rdd_dist.take(10)

[('A', 'As'),
 ('D', 'Day'),
 ('N', 'Nears'),
 ('M', 'Many'),
 ('P', 'People'),
 ('A', 'Are'),
 ('P', 'Planning'),
 ('T', 'Travel'),
 ('G', 'Gettogethers'),
 ('T', 'To')]

reduceByKey will merge the values for each key using an associative and commutative reduce function.

This will also perform the merging locally on each mapper before
sending results to a reducer, similarly to a "combiner" in MapReduce.

Primary focus is to find the words which are starting the same char.

In [ ]:
sp_rdd_unsorted = sp_rdd_dist.reduceByKey(lambda x,y : str(x)+ ', ' +str(y))

In [ ]:
sp_rdd_unsorted.count()

24

In [ ]:
sp_rdd_unsorted.take(24)

[('N', 'Nears, New, Now, Navigate'),
 ('S',
  'Same, Since, States, Should, Stay, Steps, Start, Spoke, Shes, Severe, See, Safety, Safe, Some, School, Said'),
 ('R', 'Rates, Risk, Riskto, Report, Reduce, Reason'),
 ('O', 'Of, One, Occurring, Our, Officials'),
 ('J', 'January, Journey'),
 ('W', 'Weekend, What, Want, Wen, Washington, Were, We, With, Well, Who'),
 ('C',
  'Compared, County, Centers, Covid19, Climbing, Coronavirus, Cnn, Control'),
 ('L', 'Last, Labor, Levels, Lifelines, Likely, Los'),
 ('A',
  'As, Are, Also, A, An, Author, Against, And, At, About, Affect, Analyst, Angeles'),
 ('D', 'Day, Does, Doctors, During, Drleana, Different, Disease'),
 ('M', 'Many, Members, Management, More, Medical, Milken, Main'),
 ('P',
  'People, Planning, Parts, Physician, Protect, Published, Professor, Policy, Public, Prevention'),
 ('T',
  'Travel, To, This, Time, Their, Think, Take, Times, The, Than, They, Together, These, That'),
 ('G', 'Gettogethers, George'),
 ('I', 'Is, In, If, Infections

Sorting the rdd by sortByKey method will be by desending order.

In [ ]:
sp_rdd_sorted = sp_rdd_unsorted.sortByKey(False)

In [ ]:
sp_rdd_sorted.take(24)

[('Y', 'Year'),
 ('W', 'Weekend, What, Want, Wen, Washington, Were, We, With, Well, Who'),
 ('V', 'Vaccines, Vaccinated, Visiting, Very'),
 ('U', 'Us, United, University, Unvaccinated'),
 ('T',
  'Travel, To, This, Time, Their, Think, Take, Times, The, Than, They, Together, These, That'),
 ('S',
  'Same, Since, States, Should, Stay, Steps, Start, Spoke, Shes, Severe, See, Safety, Safe, Some, School, Said'),
 ('R', 'Rates, Risk, Riskto, Report, Reduce, Reason'),
 ('Q', 'Questions'),
 ('P',
  'People, Planning, Parts, Physician, Protect, Published, Professor, Policy, Public, Prevention'),
 ('O', 'Of, One, Occurring, Our, Officials'),
 ('N', 'Nears, New, Now, Navigate'),
 ('M', 'Many, Members, Management, More, Medical, Milken, Main'),
 ('L', 'Last, Labor, Levels, Lifelines, Likely, Los'),
 ('J', 'January, Journey'),
 ('I', 'Is, In, If, Infections, It, Institute'),
 ('H',
  'Higher, Have, Hospitalizations, How, Health, Hospitalized, Holiday, Highest, House, Help, Healththings'),
 ('G', 'G

Formatting the RDD to store them in file as required using saveAsTextFile method.

In [ ]:
sp_rdd_sorted.map(lambda row: str(row[0]) + ",\t\t" +str(row[1])) \
.coalesce(1).saveAsTextFile("ICP_01_OUTPUT_01")